# Columbus Data Science Meetup
# Regression With Python and statsmodels
## Charles Carter
### November 8, 2022

## Import modules
First, we import the necessary modules. For this demonstration, I will be using Pandas for data
manipulation, and statsmodels for regression analysis.

In [1]:
import pandas as pd
import statsmodels.api as sm

## Getting the data

I will use an employee salaries dataset. I'm sorry but I do not remember the provanance
of this data so I cannot give a citation. I load the data and look at the number of rows
and columns, the data types, and the names of the variables.

In [2]:
saldf = pd.read_csv("employee_salaries.csv")
print("The shape of saldf is", saldf.shape)
print("The data types contained in saldf are",saldf.dtypes)
print("The names of the columns in saldf are", saldf.columns)

The shape of saldf is (35, 6)
The data types contained in saldf are Employee                          int64
Salary                            int64
Age                               int64
MBA                              object
Relevant Years of  Experience     int64
Number of Reports                 int64
dtype: object
The names of the columns in saldf are Index(['Employee', 'Salary', 'Age', 'MBA', 'Relevant Years of  Experience',
       'Number of Reports'],
      dtype='object')


## Renaming columns

I don't like column names with spaces, so I will rename the variable names that contain spaces.
This is merely a minor personal disfunctionality.

In [3]:
saldf = saldf.rename(columns = {
    'Relevant Years of  Experience':'Experience',
    'Number of Reports':'Reports'
})

## Change to index

Looking at the Pandas dataframe, I see both an index and an Employee number. I will make
the Employee number the index for convenience.

In [4]:
print(saldf.head())
saldf.set_index('Employee', inplace = True)
saldf.head()

   Employee  Salary  Age  MBA  Experience  Reports
0         1   28260   25   No           1        0
1         2   43392   28  Yes           4        0
2         3   56322   37  Yes          13        8
3         4   26086   23   No           0        0
4         5   36807   32   No           3        0


,Salary,Age,MBA,Experience,Reports
Employee,,,,,
1,28260,25,No,1,0
2,43392,28,Yes,4,0
3,56322,37,Yes,13,8
4,26086,23,No,0,0
5,36807,32,No,3,0


## Looking at the data

Let's look at the data. The *describe()* method returns the count, mean, standard deviation, etc.
of each variable. Unfortunately, the MBA variable isn't numeric, so we need to deal wth that.

In [6]:
saldf.describe()

,Salary,Age,Experience,Reports
count,35.000000,35.000000,35.000000,35.000000
mean,45310.285714,40.114286,9.742857,1.628571
std,13137.963289,12.266097,8.099901,2.690787
min,26086.000000,23.000000,0.000000,0.000000
25%,34527.500000,30.500000,3.500000,0.000000
50%,42377.000000,37.000000,8.000000,0.000000
75%,56053.500000,51.500000,15.000000,3.500000
max,84876.000000,61.000000,27.000000,8.000000


## Creating a dummy variable

We need to create a dummy variable for MBA, in order to use it for our regression analysis.
In effect, this will give us a 1 for MBA == True, and a 0 for MBA == False.
Notice the name change for the dummy variable.

In [7]:
print(saldf.columns)
salMBAdf = pd.get_dummies(saldf, columns = ['MBA'], drop_first = True)
salMBAdf.tail()

Index(['Salary', 'Age', 'MBA', 'Experience', 'Reports'], dtype='object')


,Salary,Age,Experience,Reports,MBA_Yes
Employee,,,,,
31,27399,24,1,0,0
32,55785,51,24,8,0
33,34649,30,5,0,0
34,64236,61,25,4,0
35,50241,45,8,0,0


## Creating the response variable

In our analysis, the salary s the response variable. We remove it from the data set and create
a Pandas series to hold the response variable. Note that the index remains the same.

In [8]:
salary = salMBAdf['Salary']
Xvars = salMBAdf.drop('Salary', axis = 1);
print(salary)
print(Xvars.columns)

Employee
1     28260
2     43392
3     56322
4     26086
5     36807
6     57119
7     48907
8     34301
9     31104
10    60054
11    41420
12    36508
13    40015
14    48329
15    39849
16    31985
17    59160
18    60335
19    35911
20    57814
21    42377
22    62430
23    46928
24    34403
25    45714
26    42247
27    54789
28    31702
29    34406
30    84876
31    27399
32    55785
33    34649
34    64236
35    50241
Name: Salary, dtype: int64
Index(['Age', 'Experience', 'Reports', 'MBA_Yes'], dtype='object')


## First attempt

We create a model using the response variable and all the predictor variables.

In [9]:
mod01 = sm.OLS(salary, Xvars).fit()
print(mod01)
mod01.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 Salary   R-squared (uncentered):                   0.997
Model:                            OLS   Adj. R-squared (uncentered):              0.996
Method:                 Least Squares   F-statistic:                              2260.
Date:                Mon, 07 Nov 2022   Prob (F-statistic):                    9.73e-38
Time:                        10:24:01   Log-Likelihood:                         -326.90
No. Observations:                  35   AIC:                                      661.8
Df Residuals:                      31   BIC:                                      668.0
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Age         1077.7113     27.447     39.265      0.000    1021.732    1133.691
Experience   -97.9963     99.993     -0.980      0.335    -301.933     105.941
Reports      277.3307    242.386      1.144      0.261    -217.019     771.680
MBA_Yes     1.461e+04   1305.252     11.195      0.000     1.2e+04    1.73e+04
==============================================================================
Omnibus:                        7.236   Durbin-Watson:                   1.859
Prob(Omnibus):                  0.027   Jarque-Bera (JB):                6.196
Skew:                           0.710   Prob(JB):                       0.0451
Kurtosis:                       4.494   Cond. No.                         115.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Second model

We don't have an intercept term, so we add one with the *add_constant()* method. We then look at
the dataframe.

In [10]:
Xvars2 = sm.add_constant(Xvars, prepend = True)
Xvars2.head()

,const,Age,Experience,Reports,MBA_Yes
Employee,,,,,
1,1.0,25,1,0,0
2,1.0,28,4,0,1
3,1.0,37,13,8,1
4,1.0,23,0,0,0
5,1.0,32,3,0,0


In [11]:
mod02 = sm.OLS(salary, Xvars2).fit()
mod02.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Salary   R-squared:                       0.956
Model:                            OLS   Adj. R-squared:                  0.950
Method:                 Least Squares   F-statistic:                     162.1
Date:                Mon, 07 Nov 2022   Prob (F-statistic):           7.42e-20
Time:                        10:24:04   Log-Likelihood:                -326.50
No. Observations:                  35   AIC:                             663.0
Df Residuals:                      30   BIC:                             670.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2261.3262   2698.459      0.838      0.409   -3249.663    7772.315
Age         1008.8343     86.695     11.637      0.000     831.779    1185.890
Experience   -47.0730    117.423     -0.401      0.691    -286.883     192.737
Reports      384.3146    274.989      1.398      0.172    -177.288     945.917
MBA_Yes     1.408e+04   1459.525      9.644      0.000    1.11e+04    1.71e+04
==============================================================================
Omnibus:                       12.866   Durbin-Watson:                   1.953
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               16.016
Skew:                           0.992   Prob(JB):                     0.000333
Kurtosis:                       5.655   Cond. No.                         245.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Third model

Let's add an interaction term, Age X MBA, and see what happens.

In [12]:
Xvars3 = Xvars
Xvars3['AgeMBA'] = Xvars['Age'] * Xvars['MBA_Yes']
Xvars3.head()

,Age,Experience,Reports,MBA_Yes,AgeMBA
Employee,,,,,
1,25,1,0,0,0
2,28,4,0,1,28
3,37,13,8,1,37
4,23,0,0,0,0
5,32,3,0,0,0


In [13]:
mod03 = sm.OLS(salary, Xvars3).fit()
mod03.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 Salary   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                              3283.
Date:                Mon, 07 Nov 2022   Prob (F-statistic):                    4.06e-40
Time:                        10:24:07   Log-Likelihood:                         -315.92
No. Observations:                  35   AIC:                                      641.8
Df Residuals:                      30   BIC:                                      649.6
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Age         1093.3985     20.616     53.038      0.000    1051.296    1135.501
Experience  -131.1555     74.552     -1.759      0.089    -283.410      21.099
Reports       13.6447    187.259      0.073      0.942    -368.788     396.078
MBA_Yes     -515.9289   3110.925     -0.166      0.869   -6869.285    5837.427
AgeMBA       450.2748     87.980      5.118      0.000     270.595     629.955
==============================================================================
Omnibus:                        0.313   Durbin-Watson:                   1.530
Prob(Omnibus):                  0.855   Jarque-Bera (JB):                0.489
Skew:                          -0.146   Prob(JB):                        0.783
Kurtosis:                       2.501   Cond. No.                         371.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Fourth model

We'll add back the intercept with the interaction term.

In [14]:
Xvars4 = Xvars2
Xvars4['AgeMBA'] = Xvars['Age'] * Xvars['MBA_Yes']
Xvars4.head()

,const,Age,Experience,Reports,MBA_Yes,AgeMBA
Employee,,,,,,
1,1.0,25,1,0,0,0
2,1.0,28,4,0,1,28
3,1.0,37,13,8,1,37
4,1.0,23,0,0,0,0
5,1.0,32,3,0,0,0


In [15]:
mod04 = sm.OLS(salary, Xvars4).fit()
mod04.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Salary   R-squared:                       0.980
Model:                            OLS   Adj. R-squared:                  0.976
Method:                 Least Squares   F-statistic:                     278.3
Date:                Mon, 07 Nov 2022   Prob (F-statistic):           1.43e-23
Time:                        10:24:11   Log-Likelihood:                -312.97
No. Observations:                  35   AIC:                             637.9
Df Residuals:                      29   BIC:                             647.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4381.4964   1900.278      2.306      0.028     494.991    8268.002
Age          961.2360     60.474     15.895      0.000     837.553    1084.919
Experience   -35.2186     81.179     -0.434      0.668    -201.249     130.812
Reports      199.2171    192.699      1.034      0.310    -194.896     593.330
MBA_Yes    -2801.6064   3072.990     -0.912      0.369   -9086.576    3483.363
AgeMBA       487.3595     83.819      5.814      0.000     315.930     658.789
==============================================================================
Omnibus:                        0.416   Durbin-Watson:                   1.746
Prob(Omnibus):                  0.812   Jarque-Bera (JB):                0.571
Skew:                          -0.149   Prob(JB):                        0.752
Kurtosis:                       2.450   Cond. No.                         403.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Fifth model

We see that the variables with the least P values are Age, MBA, and the interaction between
Age and MBA. Let's remove all other variables and see what happens.


In [16]:
newcols = ['Age', 'MBA_Yes', 'AgeMBA']
Xvars5 = Xvars2[newcols]
Xvars5.head()

,Age,MBA_Yes,AgeMBA
Employee,,,
1,25,0,0
2,28,1,28
3,37,1,37
4,23,0,0
5,32,0,0


In [17]:
mod05 = sm.OLS(salary, Xvars5).fit()
mod05.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 Salary   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                              5205.
Date:                Mon, 07 Nov 2022   Prob (F-statistic):                    4.32e-43
Time:                        10:24:24   Log-Likelihood:                         -317.92
No. Observations:                  35   AIC:                                      641.8
Df Residuals:                      32   BIC:                                      646.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Age         1058.4420      9.622    110.003      0.000    1038.843    1078.041
MBA_Yes      931.4293   3017.673      0.309      0.760   -5215.370    7078.228
AgeMBA       414.7154     84.353      4.916      0.000     242.893     586.537
==============================================================================
Omnibus:                        0.224   Durbin-Watson:                   1.762
Prob(Omnibus):                  0.894   Jarque-Bera (JB):                0.205
Skew:                          -0.159   Prob(JB):                        0.903
Kurtosis:                       2.801   Cond. No.                         339.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""